<a href="https://colab.research.google.com/github/S-Kaito/s-kaito.github.io/blob/master/notebook/seminar/2020_0612.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import struct
import numpy as np
import codecs
import math
import itertools
from PIL import Image
import numpy as np

def read_picture(path):
    """
    Read picture from file

    Parameter
    ---------
    path: string
        file path
    
    return
    ------
    size: (width: int, height: int)
        picture size
    max_n: int
        max value
    data: NdArray<NdArray<int>>
        picture data
    """
    
    # 画像を開く(Header情報用)
    infile = codecs.open(path, 'r', "utf-8", "ignore")
    # 画像を開く(画像データ用)
    img = np.array(Image.open(path))

    infile.readline()
    infile.readline()
    width, height = map(int, infile.readline().split())
    max_n = int(infile.readline())

    infile.close()
    return (width, height), max_n, img

def write_picture(path, size, n, data):
    """
    Write picture to file
    Parameter
    ---------
    path: string
        file path
    size: (width: int, height: int)
        picture size
    n: int
        max value
    data: Array<Array<[r: int, g: int, b: int]>>
        picture data
    """

    # 画像を書き出す
    pilImg = Image.fromarray(np.uint8(data.reshape((size[0], size[1]))))
    pilImg.save(path)

def parse_gray_scale(data):
    # NTSC形式用の配列
    value = np.array([
        [0.299, 0.587, 0.114],
        [0.596, -0.274, -0.322],
        [0.212, -0.523, 0.311]
    ])

    data = data.reshape((65536, 3))

    #行列の掛け算を行い、そのうちのYのみで配列を生成する
    res = [math.floor(np.dot(value, d)[0]) for d in data]

    return np.array(res).reshape((256, 256))

def edge_extraction(img, type="roberts"):
    filter = [[8], [8]]
    size = (1, 1)
    if type == "roberts": # Robertマトリクス
        filter = np.array([[[ 0, 0, 0],
                            [ 0, 0, 1],
                            [ 0,-1, 0]],
                           [[ 0, 0, 0],
                            [ 0, 1, 0],
                            [ 0, 0,-1]]])
        size = (3, 3)
    elif type == "prewitt": # Prewittマトリクス
        filter = np.array([[[-1, 0, 1],
                            [-1, 0, 1],
                            [-1, 0, 1]],
                           [[-1,-1,-1],
                            [ 0, 0, 0],
                            [ 1, 1, 1]]])
        size = (3, 3)
    elif type == "sobel": # Sobelマトリクス
        filter = np.array([[[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]],
                           [[-1,-2,-1],
                            [ 0, 0, 0],
                            [ 1, 2, 1]]])
        size = (3, 3)

    shape = img.shape

    ni = np.copy(img)

    for i in range(1, shape[0] - 1):
        for j in range(1, shape[1] - 1):

            # x,y方向それぞれの勾配を求める
            dx = math.floor(np.sum(img[i - size[0] // 2: i + size[0] // 2 + 1, j - size[1] // 2: j + size[1] // 2 + 1] * filter[0]))
            dy = math.floor(np.sum(img[i - size[0] // 2: i + size[0] // 2 + 1, j - size[1] // 2: j + size[1] // 2 + 1] * filter[1]))

            # 方向に依存しない形に変換
            val = math.sqrt(dx ** 2 + dy ** 2)
            ni[i][j] = max(0, min(255, val))

    return ni

def border_tracing(img):
    img = img.T.copy()

    # 探索の方向を定義
    pri = [[-1,-1], [-1, 0], [-1, 1], [ 0, 1], [1, 1], [ 1, 0], [ 1, -1], [0, -1]]

    shape = img.shape

    # 画像の端を背景色とした画像を生成
    zp = np.zeros((shape[0] + 2, shape[1] + 2))
    zp[1: -1, 1: -1] = img[:,:]

    ni = np.zeros(shape)

    last = 0

    for j in range(shape[1]):
        for i in range(shape[0]):
            # 直前の画素が背景色か？
            if(last == 255 and img[i][j] == 255):
                continue

            last = img[i][j]

            # 現在の画素が背景色でないか、すでに探索済みでないか？
            if img[i][j] == 0 or ni[i][j] == 255:
                continue

            # 孤立点か？
            if (np.all(zp[i: i + 3, j: j + 3] == [[0, 0, 0], [0, 255, 0], [0, 0, 0]])):
                ni[i][j] = 255
                continue

            p = 2
            x = i
            y = j
            frag = True

            ni[i][j] = 255
            zp[i + 1][j + 1] = 64

            # 開始点に戻るか8方向すべてのチェックが行われるまでループ            
            while frag:
                # 8方向それぞれの確認を行う
                for k in range(0, 8):
                    xx = x + pri[p][0]
                    yy = y + pri[p][1]

                    # 探索開始点か？
                    if(xx == i and yy == j):
                        frag = False
                        break
                    
                    # 背景色でないか？
                    if zp[xx + 1][yy + 1] != 0:
                    
                        # 新しい画像に白を代入し、探索済みのチェックを入れる
                        ni[xx][yy] = 255
                        zp[xx + 1][yy + 1] = 128

                        if p == 0 or p == 7:
                            p = 6
                        elif p == 1 or p == 2:
                            p = 0
                        elif p == 3 or p == 4:
                            p = 2
                        elif p == 5 or p == 6:
                            p = 4
                        x = xx
                        y = yy
                        break
                    else:
                        p = (p + 1) % 8
                    
                else:
                    break
    return ni.T.copy()

In [0]:
PATH = "girl.ppm"
PATH_2 = "star2.pgm"
size, n, img = read_picture(PATH)

# 画像をグレイスケールに変換
img = parse_gray_scale(img)

# Rovertsでエッジ抽出
img_e = edge_extraction(img)
write_picture("girl_e.pgm", size, n, img_e)

# Prewittでエッジ抽出
img_ep = edge_extraction(img, type="prewitt")
write_picture("girl_ep.pgm", size, n, img_ep)

# Sobelでエッジ抽出
img_es = edge_extraction(img, type="sobel")
write_picture("girl_es.pgm", size, n, img_es)

# 境界線追跡でエッジ抽出
size_b, n_b, img_b = read_picture(PATH_2)
img_bt = border_tracing(img_b)
write_picture("star2_bt.pgm", size, n, img_bt)

print("Finish!!")

Finish!!
